# batch normalization이 미치는 영향

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [3]:
torch.manual_seed(1)


In [4]:
mnist_train = datasets.MNIST(root='MNIST_data/', train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root='MNIST_data/', train=False, download=True, transform=transforms.ToTensor())

C:\Users\DMQA\anaconda3\envs\main\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
train_loader = DataLoader(dataset=mnist_train, batch_size=32, shuffle=True, drop_last=True)
train_test = DataLoader(dataset=mnist_test, batch_size=32, shuffle=True, drop_last=True)

In [30]:
class basic_model(nn.Module):
    def __init__(self):
        super(basic_model, self).__init__()
        self.fc1 = nn.Linear(784, 32, bias=True)
        self.fc2 = nn.Linear(32, 32, bias=True)
        self.fc3 = nn.Linear(32, 10, bias=True)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        print(out)
        out = self.relu(out)
        print(out)
        out = self.fc2(out)
        print(out)
        out = self.relu(out)
        print(out)
        out = self.fc3(out)
        return out

class batchnorm_model(nn.Module):
    def __init__(self):
        super(batchnorm_model,self).__init__()
        self.fc1 = nn.Linear(784, 32, bias=True)
        self.fc2 = nn.Linear(32, 32, bias=True)
        self.fc3 = nn.Linear(32, 10, bias=True)
        self.relu = nn.ReLU()
        self.batch1 = nn.BatchNorm1d(32)
        self.batch2 = nn.BatchNorm1d(32)
    
    def forward(self, x):
        out = self.fc1(x)
        print(out)
        out = self.batch1(out)
        print(out)
        out = self.relu(out)
        print(out)
        out = self.fc2(out)
        print(out)
        out = self.batch2(out)
        print(out)
        out = self.relu(out)
        print(out)
        out = self.fc3(out)
        return out

In [20]:
base_model = basic_model()
base_criterion = torch.nn.CrossEntropyLoss()
base_optimizer = optim.Adam(base_model.parameters(), lr=0.001)

base_train_loss = []
base_test_loss = []

for epoch in range(1):
    base_model.train()

    for X,Y in train_loader:
        X = X.view(-1, 28*28)
        Y = Y

        base_optimizer.zero_grad()
        base_predict = base_model(X)
        base_loss = base_criterion(base_predict, Y)
        base_loss.backward()
        base_optimizer.step()

        break
        

tensor([[ 0.0985,  0.3523, -0.1310,  ..., -0.0348,  0.0980, -0.1088],
        [-0.0687,  0.4167,  0.1739,  ...,  0.4393, -0.3086, -0.1690],
        [ 0.1127,  0.1878, -0.1721,  ..., -0.0594, -0.2055,  0.1464],
        ...,
        [ 0.2664,  0.3396,  0.0141,  ..., -0.0230, -0.2620,  0.1863],
        [-0.0839,  0.3810, -0.0706,  ...,  0.1994, -0.2350, -0.3610],
        [ 0.2247,  0.0920, -0.0913,  ...,  0.1120, -0.1500, -0.0839]],
       grad_fn=<AddmmBackward>)
tensor([[0.0985, 0.3523, 0.0000,  ..., 0.0000, 0.0980, 0.0000],
        [0.0000, 0.4167, 0.1739,  ..., 0.4393, 0.0000, 0.0000],
        [0.1127, 0.1878, 0.0000,  ..., 0.0000, 0.0000, 0.1464],
        ...,
        [0.2664, 0.3396, 0.0141,  ..., 0.0000, 0.0000, 0.1863],
        [0.0000, 0.3810, 0.0000,  ..., 0.1994, 0.0000, 0.0000],
        [0.2247, 0.0920, 0.0000,  ..., 0.1120, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)
tensor([[-0.1531, -0.0840, -0.0233,  ...,  0.1201, -0.0629, -0.1779],
        [-0.2691, -0.0878, -0.0110

In [31]:
bn_model = batchnorm_model()
bn_criterion = torch.nn.CrossEntropyLoss()
bn_optimizer = optim.Adam(bn_model.parameters(), lr=0.001)

bn_train_loss = []
bn_test_loss = []

for epoch in range(1):
    bn_model.train()

    for X,Y in train_loader:
        X = X.view(-1, 28*28)
        Y = Y

        bn_optimizer.zero_grad()
        bn_predict = bn_model(X)
        bn_loss = bn_criterion(bn_predict, Y)
        bn_loss.backward()
        bn_optimizer.step()

        break

tensor([[-0.0755, -0.1436,  0.2441,  ...,  0.0120,  0.0356,  0.1408],
        [-0.0237, -0.0166, -0.0407,  ...,  0.2503, -0.0739, -0.1568],
        [-0.2325, -0.2433,  0.0261,  ...,  0.1156,  0.1225, -0.3519],
        ...,
        [-0.1136, -0.1849,  0.5610,  ..., -0.2251,  0.0367,  0.0197],
        [ 0.2148, -0.1487, -0.1223,  ...,  0.1893,  0.0122, -0.1701],
        [ 0.0588,  0.0563, -0.0128,  ...,  0.1446,  0.0223, -0.1234]],
       grad_fn=<AddmmBackward>)
tensor([[-0.9007,  0.1805,  1.1894,  ..., -0.0174, -0.8748,  1.9486],
        [-0.5688,  1.1176, -0.5197,  ...,  1.4355, -1.5931, -0.0503],
        [-1.9067, -0.5546, -0.1189,  ...,  0.6144, -0.3049, -1.3604],
        ...,
        [-1.1445, -0.1235,  3.0915,  ..., -1.4625, -0.8677,  1.1352],
        [ 0.9595,  0.1430, -1.0095,  ...,  1.0637, -1.0279, -0.1392],
        [-0.0398,  1.6550, -0.3523,  ...,  0.7911, -0.9618,  0.1741]],
       grad_fn=<NativeBatchNormBackward>)
tensor([[0.0000, 0.1805, 1.1894,  ..., 0.0000, 0.0000, 1.9

In [2]:
bn_model = batchnorm_model()
bn_criterion = torch.nn.CrossEntropyLoss()
bn_optimizer = optim.Adam(bn_model.parameters(), lr=0.001)

bn_train_loss = []
bn_test_loss = []


0
